In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
# sc = pyspark.SparkContext(master = "spark://spark-master:7077",appName="Drowsiness")
# Create Spark session
master = "spark://spark-master:7077"
appName = "Drowsiness"
spark = SparkSession.builder.master(master).appName(appName).getOrCreate()

In [3]:
#load df file
df = spark.read.format('csv').option('header',True).option('multiLine', True).load('file:///jupyter-data/data/total.csv')
df.show()
print(f'Record count is: {df.count()}')

+-------------------+------------------+-------------------+------------------+---+----------+
|                EAR|               MAR|        Circularity|               MOE|  Y|respondent|
+-------------------+------------------+-------------------+------------------+---+----------+
| 0.2746267554059725|0.7452161377294898|  0.366795128235752|2.7135598519083066|0.0|         4|
| 0.2508982799024048|0.7746643284693926| 0.4021396462592153|3.0875633295322857|0.0|         4|
|0.30037087831054704|0.6885045109064004| 0.4150342711218802| 2.292181301925582|0.0|         4|
| 0.2935651535998499|0.7048471326472809| 0.4489237073878338| 2.400990458179644|0.0|         4|
| 0.2674082098871901| 0.766533212092524| 0.4210985139768859|2.8665283403822817|0.0|         4|
| 0.2754654478027501|0.7692327850320306| 0.4476255954755593|2.7924837440332904|0.0|         4|
| 0.2993355473569827| 0.764556021941758| 0.4689933815535661|2.5541771723822726|0.0|         4|
|0.27569820671823103|0.8157975682555313| 0.4675241

In [5]:
df.dtypes

[('EAR', 'string'),
 ('MAR', 'string'),
 ('Circularity', 'string'),
 ('MOE', 'string'),
 ('Y', 'string'),
 ('respondent', 'string')]

In [7]:
#Change data type
from functools import reduce
changedTypedf = (reduce(
    lambda memo_df, col_name: 
        memo_df.withColumn(col_name, memo_df[col_name].cast('float')),
    ["EAR", "MAR","Circularity","MOE","Y","respondent"],df))

Record count is: 17925


In [9]:
changedTypedf.dtypes
changedTypedf.show()

+----------+----------+-----------+---------+---+----------+
|       EAR|       MAR|Circularity|      MOE|  Y|respondent|
+----------+----------+-----------+---------+---+----------+
|0.27462676|0.74521613| 0.36679512|2.7135599|0.0|       4.0|
|0.25089827|0.77466434| 0.40213963|3.0875633|0.0|       4.0|
|0.30037087| 0.6885045| 0.41503426|2.2921813|0.0|       4.0|
|0.29356515|0.70484716|  0.4489237|2.4009905|0.0|       4.0|
|0.26740822| 0.7665332|  0.4210985|2.8665283|0.0|       4.0|
|0.27546546| 0.7692328|  0.4476256|2.7924838|0.0|       4.0|
|0.29933554|0.76455605|  0.4689934|2.5541773|0.0|       4.0|
| 0.2756982|0.81579757| 0.46752414| 2.959024|0.0|       4.0|
|0.27462676|0.81600577|  0.4030637|2.9713266|0.0|       4.0|
|0.31206918|0.77780604| 0.45223483|2.4924154|0.0|       4.0|
|  0.274051|  0.833626| 0.40965688|3.0418644|0.0|       4.0|
|      0.25| 0.8246894|  0.3588233|3.2987576|0.0|       4.0|
|0.25118446|0.75357527| 0.40748096| 3.000087|0.0|       4.0|
| 0.3164909|  0.814271| 

In [10]:
import pandas as pd
import numpy as np

pddf = changedTypedf.toPandas()
spark.stop()

In [12]:
pddf

,EAR,MAR,Circularity,MOE,Y,respondent
0,0.274627,0.745216,0.366795,2.713560,0.0,4.0
1,0.250898,0.774664,0.402140,3.087563,0.0,4.0
2,0.300371,0.688505,0.415034,2.292181,0.0,4.0
3,0.293565,0.704847,0.448924,2.400990,0.0,4.0
4,0.267408,0.766533,0.421099,2.866528,0.0,4.0
...,...,...,...,...,...,...
17920,0.348480,1.049561,0.490477,3.011821,5.0,60.0
17921,0.262352,1.053755,0.395392,4.016572,5.0,60.0
17922,0.319431,1.026230,0.490811,3.212684,5.0,60.0
17923,0.256146,1.083715,0.378786,4.230846,5.0,60.0


In [16]:
#Separating the rows which are "Alert" only
df_alert = pddf[pddf["Y"]==0.0]

In [18]:
df_alert

,EAR,MAR,Circularity,MOE,Y,respondent
0,0.274627,0.745216,0.366795,2.713560,0.0,4.0
1,0.250898,0.774664,0.402140,3.087563,0.0,4.0
2,0.300371,0.688505,0.415034,2.292181,0.0,4.0
3,0.293565,0.704847,0.448924,2.400990,0.0,4.0
4,0.267408,0.766533,0.421099,2.866528,0.0,4.0
...,...,...,...,...,...,...
17442,0.291844,1.016871,0.461614,3.484300,0.0,60.0
17443,0.273138,1.004210,0.436752,3.676572,0.0,60.0
17444,0.289664,1.017200,0.453121,3.511660,0.0,60.0
17445,0.291827,1.033636,0.451550,3.541943,0.0,60.0


In [20]:
#Creating separate dataframes for each respondent's first, second and third "Alert" frame
df_alert_1 = df_alert.iloc[0::240, :]
df_alert_2 = df_alert.iloc[1::240, :]
df_alert_3 = df_alert.iloc[2::240, :]
print(df_alert_1)
print(df_alert_2)
print(df_alert_3)

            EAR       MAR  Circularity       MOE    Y  respondent
0      0.274627  0.745216     0.366795  2.713560  0.0         4.0
718    0.357240  0.969858     0.485638  2.714863  0.0         6.0
1436   0.373533  1.220598     0.560824  3.267715  0.0         9.0
2154   0.261831  1.028812     0.432978  3.929305  0.0        11.0
2872   0.342611  1.057268     0.450888  3.085916  0.0        12.0
3590   0.297326  1.076569     0.456573  3.620838  0.0        15.0
4308   0.376992  1.217219     0.529888  3.228765  0.0        16.0
5026   0.365051  1.271549     0.476183  3.483205  0.0        17.0
5744   0.419664  1.019130     0.593993  2.428441  0.0        23.0
6462   0.286589  0.963140     0.397076  3.360703  0.0        24.0
7180   0.325767  1.228886     0.499502  3.772282  0.0        27.0
7898   0.224534  1.033715     0.384983  4.603833  0.0        29.0
8616   0.256629  0.993606     0.387675  3.871752  0.0        34.0
9334   0.300926  1.061985     0.458458  3.529055  0.0        35.0
10052  0.2

In [21]:
#Merging them into one dataframe
alert_first3 = [df_alert_1,df_alert_2,df_alert_3]
df_alert_first3 = pd.concat(alert_first3)
df_alert_first3 = df_alert_first3.sort_index()
print(df_alert_first3)

            EAR       MAR  Circularity       MOE    Y  respondent
0      0.274627  0.745216     0.366795  2.713560  0.0         4.0
1      0.250898  0.774664     0.402140  3.087563  0.0         4.0
2      0.300371  0.688505     0.415034  2.292181  0.0         4.0
718    0.357240  0.969858     0.485638  2.714863  0.0         6.0
719    0.329274  1.008089     0.455214  3.061552  0.0         6.0
...         ...       ...          ...       ...  ...         ...
16515  0.294118  0.905203     0.467201  3.077691  0.0        55.0
16516  0.333333  0.889020     0.468848  2.667060  0.0        55.0
17232  0.308928  0.967144     0.457839  3.130648  0.0        60.0
17233  0.268798  1.000000     0.393098  3.720260  0.0        60.0
17234  0.273833  0.959964     0.395433  3.505657  0.0        60.0

[75 rows x 6 columns]


In [24]:
#Based on the first 3 "Alert" frames, calculating respondent-wise mean and std for each feature
df_means = df_alert_first3.groupby("respondent")[["EAR","MAR","Circularity","MOE"]].mean()
df_std = df_alert_first3.groupby("respondent")[["EAR","MAR","Circularity","MOE"]].std()

In [25]:
print(df_means,df_std)

                 EAR       MAR  Circularity       MOE
respondent                                           
4.0         0.275299  0.736128     0.394656  2.697768
6.0         0.339332  0.985335     0.466794  2.908994
9.0         0.305310  1.205328     0.493971  4.080558
11.0        0.251610  1.027279     0.425526  4.173456
12.0        0.317457  1.048695     0.432872  3.335379
15.0        0.253946  1.014731     0.388025  4.091606
16.0        0.376435  1.189179     0.529919  3.161091
17.0        0.346792  1.254015     0.474040  3.624841
23.0        0.414416  0.969532     0.578720  2.343089
24.0        0.293527  1.000341     0.406376  3.421671
27.0        0.324521  1.164619     0.504930  3.603816
29.0        0.186747  1.018197     0.365557  5.754826
34.0        0.278620  0.985476     0.419529  3.549783
35.0        0.305254  1.044204     0.461889  3.424570
36.0        0.278180  0.961090     0.436460  3.461294
37.0        0.239422  1.074378     0.386591  4.562799
39.0        0.286569  1.0952

In [26]:
#Functions for mean and std for each feature
def mean_EAR(respondent):
    return df_means.loc[respondent]["EAR"]

def mean_MAR(respondent):
    return df_means.loc[respondent]["MAR"]

def mean_Circularity(respondent):
    return df_means.loc[respondent]["Circularity"]

def mean_MOE(respondent):
    return df_means.loc[respondent]["MOE"]


def std_EAR(respondent):
    return df_std.loc[respondent]["EAR"]

def std_MAR(respondent):
    return df_std.loc[respondent]["MAR"]

def std_Circularity(respondent):
    return df_std.loc[respondent]["Circularity"]

def std_MOE(respondent):
    return df_std.loc[respondent]["MOE"]

In [27]:
#Adding respondent-wise mean and std for each feature to each row in the original dataframe
pddf["EAR_mean"] = pddf["respondent"].apply(mean_EAR)
pddf["MAR_mean"] = pddf["respondent"].apply(mean_MAR)
pddf["Circularity_mean"] = pddf["respondent"].apply(mean_Circularity)
pddf["MOE_mean"] = pddf["respondent"].apply(mean_MOE)

pddf["EAR_std"] = pddf["respondent"].apply(std_EAR)
pddf["MAR_std"] = pddf["respondent"].apply(std_MAR)
pddf["Circularity_std"] = pddf["respondent"].apply(std_Circularity)
pddf["MOE_std"] = pddf["respondent"].apply(std_MOE)


In [30]:
#Calculating normalised features for each row in the original dataframe
pddf["EAR_N"] = (pddf["EAR"]-pddf["EAR_mean"])/ pddf["EAR_std"]
pddf["MAR_N"] = (pddf["MAR"]-pddf["MAR_mean"])/ pddf["MAR_std"]
pddf["Circularity_N"] = (pddf["Circularity"]-pddf["Circularity_mean"])/ pddf["Circularity_std"]
pddf["MOE_N"] = (pddf["MOE"]-pddf["MOE_mean"])/ pddf["MOE_std"]

In [31]:
#Saving the file to a CSV with all the information
pddf.to_csv('./data/totalwithallinfo.csv',index=False)

#Saving the file to a CSV with main the information
df_main = pddf.drop(["EAR_mean","MAR_mean","Circularity_mean","MOE_mean","EAR_std","MAR_std","Circularity_std","MOE_std"],axis=1)
df_main.to_csv('./data/totalwithmaininfo.csv',index=False)